In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from langchain_groq import ChatGroq
from langchain_openai import AzureChatOpenAI
import numpy as np
import streamlit as st

In [ ]:
# API Configuration
openai_api_key = "YOUR_AZURE_OPENAI_API_KEY"
GROQ_API_KEY = "YOUR_GROQ_API_KEY"

In [ ]:
# Initialize LLM
llm_llama70b = ChatGroq(model_name="llama3-70b-8192", groq_api_key=GROQ_API_KEY)

In [ ]:
# Stock Analysis Tools
@tool
def get_basic_stock_info(ticker: str) -> pd.DataFrame:
    """Retrieves basic information about a single stock."""
    stock = yf.Ticker(ticker)
    info = stock.info
    
    basic_info = pd.DataFrame({
        'Name': [info.get('longName', 'N/A')],
        'Sector': [info.get('sector', 'N/A')],
        'Industry': [info.get('industry', 'N/A')],
        'Market Cap': [info.get('marketCap', 'N/A')],
        'Current Price': [info.get('currentPrice', 'N/A')]
    })
    return basic_info

In [ ]:
# Create Stock Analysis Agents
stock_researcher = Agent(
    llm=AzureChatOpenAI(
        model_name="gpt-4o", 
        api_key="YOUR_AZURE_OPENAI_API_KEY", 
        api_version="2024-10-21", 
        azure_endpoint="YOUR_AZURE_ENDPOINT"
    ),
    role="Stock Researcher",
    goal="Identify and analyze stock information",
    backstory="Expert stock researcher",
    tools=[get_basic_stock_info],
    verbose=True
)

In [ ]:
# Example Usage
# Replace with your actual API keys and run the analysis
print("Stock Analysis AI Agents Ready!")
print("Add your API keys to run live analysis")